In [1]:
import jax.numpy as jnp
import sys
import os
from typing import Sequence
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))

from gradient_descendent import gradient_descendent

# Tarefa 1: Otimização unimodal sem restrição

## Parte 1: Otimização analítica

### Problema

Seja a função de custo quadrática dada por: 
$$J(x) = (x - c)^T A (x - c) + b$$
onde $A$ é uma matriz. Encontre analiticamente o ponto $x^* \in \mathbb{R}^2$ que minimize a função de custo.

### Solução

Temos que $\frac{\partial {((x - c)^T A (x - c))}}{\partial x} = (A^T + A)(x - c)$ 

ou 

$\frac{\partial {((x - c)^T A (x - c))}}{\partial x} = 2A(x - c)$ no caso de $A$ ser simétrica.

Portanto, derivando a função de custo e igualando a zero, temos:
$$J_x = (A^T + A)(x - c)$$

$$J_x = (A^T + A)(x - c) = 0$$

$$J_x = (A^T + A)^{-1}(A^T + A)(x - c) = (A^T + A)^{-1} \dot \quad 0$$

$$J_x = I(x - c) = (A^T + A)^{-1} \dot \quad 0$$

$$J_x = x - c = 0$$

$$J_x = x = c$$

$$x^* = c$$

$$J_{xx} = (A^T + A)$$

Assim, se $(A^T + A) > 0$, $x^* = c$ será um ponto de minímo.

No caso de a ser simétrica

$$J_x = 2A(x - c)$$

$$J_x = 2A(x - c) = 0$$

$$J_x = 2A^{-1}A(x - c) = A^{-1}\dot \quad 0$$

$$J_x = 2(x - c) = 0$$

$$J_x = x - c = 0$$

$$J_x = x = c$$

$$J_{xx} = 2A$$

Assim, se $2A > 0$, $x^* = c$ será um ponto de minímo.

## Parte 2: Otimização numérica por gradiente descendente

### Definir a função de custo

In [2]:
def wrapper(A: Sequence[Sequence[float | int]],
            b: float | int,
            c: Sequence[float | int]):
    A_np = jnp.array(A, dtype=jnp.float32)
    b_np = jnp.float32(b)
    c_np = jnp.array(c, dtype=jnp.float32)

    def cost_function(x: jnp.ndarray) -> jnp.ndarray:
        x_np = jnp.array(x, dtype=jnp.float32)
        diff_x_c = x_np - c_np

        # (x - c)^T A (x - c) + b
        return diff_x_c.T @ A_np @ diff_x_c + b_np

    return cost_function

### Calcular o ponto mínimo usando gradiente descendente para diferentes taxas de aprendizado

In [3]:
A = [[4, 0],
     [1, 2]]
b = 2.0
c = [0, 1]

f = wrapper(A, b, c)

x = [3, 3]
learning_rates = [0.1, 0.01, 0.001, 0.3]
max_iter = 1000
tolerance = 1e-6

dict_of_results = {}

for learning_rate in learning_rates:
     x_values, costs, num_iter = gradient_descendent(x, f, learning_rate, max_iter, tolerance, 2)
     x_values = jnp.array(x_values)
     costs = jnp.array(costs)

     dict_of_results[learning_rate] = (x_values, costs)

     print(f"Taxa de aprendizado: {learning_rate}")
     print(f"Ponto mínimo: {x_values[-1]}, ou aproximadamente {jnp.round(x_values[-1], 5)}")
     print(f"Custo mínimo: {costs[-1]}, ou aproximadamente {jnp.round(costs[-1], 5)}")
     print(f"Numero de iterações: {num_iter}\n")
     print(f"{'-' * 100}\n")

Taxa de aprendizado: 0.1
Ponto mínimo: [-3.3170699e-07  1.0000014e+00], ou aproximadamente [-0.  1.]
Custo mínimo: 2.0, ou aproximadamente 2.0
Numero de iterações: 29

----------------------------------------------------------------------------------------------------

Taxa de aprendizado: 0.01
Ponto mínimo: [-6.0139814e-06  1.0000255e+00], ou aproximadamente [-9.9999997e-06  1.0000299e+00]
Custo mínimo: 2.0, ou aproximadamente 2.0
Numero de iterações: 281

----------------------------------------------------------------------------------------------------

Taxa de aprendizado: 0.001
Ponto mínimo: [-0.00580975  1.0283763 ], ou aproximadamente [-0.00581    1.0283799]
Custo mínimo: 2.0015804767608643, ou aproximadamente 2.001579999923706
Numero de iterações: 1000

----------------------------------------------------------------------------------------------------

Taxa de aprendizado: 0.3
Ponto mínimo: [nan nan], ou aproximadamente [nan nan]
Custo mínimo: nan, ou aproximadamente nan
Nume

### Definir função para plotar a função de custo e o ponto mínimo encontrado

In [4]:
def plot_function(f, x_values, costs, title="Superfície da Função de Custo + Pontos Encontrados"):
    x1_data = jnp.linspace(-4, 4, 100)
    x2_data = jnp.linspace(-4, 4, 100)
    X1, X2 = jnp.meshgrid(x1_data, x2_data)
    Y = jnp.array([[f(jnp.array([x1, x2])) for x2 in x2_data] for x1 in x1_data])


    fig = go.Figure()

    # Superfície
    fig.add_trace(go.Surface(
        x=X1,
        y=X2,
        z=Y,
        colorscale="Viridis",
        opacity=0.8
    ))

    # Pontos encontrados
    fig.add_trace(go.Scatter3d(
        x=x_values[:, 0],
        y=x_values[:, 1],
        z=costs,
        mode="markers+text",
        text=[f"P{i}" for i in range(len(x_values))],
        textposition="top center",
        marker=dict(size=6, color="red", symbol="circle")
    ))

    # Layout
    fig.update_layout(
        scene=dict(
            xaxis_title="x1",
            yaxis_title="x2",
            zaxis_title="J(x)"
        ),
        title=title,
        autosize=True,
    )

    fig.show()

### Plotar a função de custo e o ponto mínimo encontrado para diferentes taxas de aprendizado

In [5]:
x_values, costs = dict_of_results[learning_rates[0]]

plot_function(f, x_values,costs, title=f"Superfície da Função de Custo + Pontos Encontrados (Taxa de Aprendizado = {learning_rates[0]})")

In [6]:
x_values, costs = dict_of_results[learning_rates[1]]

plot_function(f, x_values,costs, title=f"Superfície da Função de Custo + Pontos Encontrados (Taxa de Aprendizado = {learning_rates[1]})")

In [7]:
x_values, costs = dict_of_results[learning_rates[2]]

plot_function(f, x_values,costs, title=f"Superfície da Função de Custo + Pontos Encontrados (Taxa de Aprendizado = {learning_rates[2]})")

In [8]:
x_values, costs = dict_of_results[learning_rates[3]]

plot_function(f, x_values,costs, title=f"Superfície da Função de Custo + Pontos Encontrados (Taxa de Aprendizado = {learning_rates[3]})")